In [2]:
import pandas as pd

In [1]:
Fitbit_dir = "./Fitbit_05042022/ShiYangLee/Physical Activity/heart_rate-2022-03-29.json"

In [3]:
df_fitbit = pd.read_json(Fitbit_dir)

In [11]:
df_fitbit.head()

,dateTime,value
0,2022-03-28 23:00:06,"{'bpm': 56, 'confidence': 3}"
1,2022-03-28 23:00:21,"{'bpm': 56, 'confidence': 3}"
2,2022-03-28 23:00:31,"{'bpm': 57, 'confidence': 3}"
3,2022-03-28 23:00:41,"{'bpm': 58, 'confidence': 3}"
4,2022-03-28 23:00:46,"{'bpm': 57, 'confidence': 3}"


In [5]:
df_fitbit.value.dtype

dtype('O')

In [6]:
df_fitbit.isnull().sum()

dateTime    0
value       0
dtype: int64

In [22]:
def data_split(data):
    data['value'] = data['value'].astype('str')
    print(data.value.dtype)
    tmp = data.value.str.split(' ', expand = True)
    data[['bpm', 'confidence']] = tmp[[

In [23]:
data_split(df_fitbit)

object
             0    1              2   3
0      {'bpm':  56,  'confidence':  3}
1      {'bpm':  56,  'confidence':  3}
2      {'bpm':  57,  'confidence':  3}
3      {'bpm':  58,  'confidence':  3}
4      {'bpm':  57,  'confidence':  3}
...        ...  ...            ...  ..
11351  {'bpm':  75,  'confidence':  3}
11352  {'bpm':  79,  'confidence':  2}
11353  {'bpm':  73,  'confidence':  2}
11354  {'bpm':  68,  'confidence':  3}
11355  {'bpm':  64,  'confidence':  3}

[11356 rows x 4 columns]
